In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
from hemul.cipher import Ring, Parameters, Context 
from hemul.scheme import Evaluator, Encryptor, Decryptor, Encoder

## Test configuration

In [2]:
abs_tol = 1e-5 # error tolerance

## Context set up

In [3]:
myring = Ring(seed=1234)
parms = Parameters(logp = 30, logq = 150, logn = 12)
context = Context(parms, myring)

sk = context.generate_secret_key()

print(context.params)

 logp: 30
 logq: 150 
 logn: 12


## Create Agents

In [4]:
keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}

ev = Evaluator(keys, context)
encryptor = Encryptor(context)

decryptor = Decryptor(sk)

encoder = Encoder(context)

## Create a ctxt and attempt to print it 
Both ndarray and list are OK

In [5]:
ctxt1 = encryptor.encrypt(np.array([1,2,3,4]))
ctxt2 = encryptor.encrypt([2,3,3,5])
print(ctxt1)

You can't read the content


#### You need to first decrypt it

In [6]:
decrypted = decryptor.decrypt(ctxt2)
print(decrypted)

[2. 3. 3. ... 0. 0. 0.]


## Addition

In [7]:
new_ctxt = ev.add(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(new_ctxt))

[3. 5. 6. ... 0. 0. 0.]


### Mult

In [8]:
ctxt3 = ev.mult(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(ctxt3))

ev.mult(ctxt1, ctxt2, inplace=True)
print(decryptor.decrypt(ctxt1))

[2. 6. 9. ... 0. 0. 0.]
[2. 6. 9. ... 0. 0. 0.]


### mult by plain

ptxt can be a
1. scalar
2. single-element sequence 
3. n_element-long sequence
4. nslots-long sequence

In [9]:
ptxt = encoder.encode([5])
ev.mult_by_plain(ctxt1, ptxt, inplace=True)

In [10]:
ev.rescale_next(ctxt1) # match scale
ev.mult_by_plain(ctxt1, ptxt, inplace=True)
print(decryptor.decrypt(ctxt1))

[50.  0.  0. ...  0.  0.  0.]


## Algorithms
### sum

In [11]:
from hemul.algorithms import Algorithms 
algo = Algorithms(ev, encoder)

summed = algo.sum_reduce(ctxt1, partial=False)

_dec = decryptor.decrypt(summed) # if partial=True, only the first slot if valid.
assert np.isclose(np.sum(ctxt1._arr), _dec[0], atol=abs_tol), "Error!"

from hemul.stats import Statistics
hest = Statistics(ev, encoder)

In [14]:
def sum_reduce(ctxt):
    """minimum sum"""
    ctxt_ = ev.copy(ctxt)
    for i in range(int(np.log2(len(ctxt)))):
        tmp = ev.copy(ctxt_)
        ev.lrot(tmp, 2**i, inplace=True)
        ev.add(ctxt_, tmp, inplace=True)   
    return ctxt_
def mean(ctxt):
    """mean without error check"""
    n = algo.encode_repeat(ctxt._n_elements)
    summed = sum_reduce(ctxt)
    return ev.div_by_plain(summed, n)
def check(ctxt):
    print(f"Check scale: {ctxt.logp},  noise budget:{ctxt.logq}")

일반코드 

In [15]:
data = [1,2,3,4]

In [18]:
m = np.mean(data)
result = np.sum((data - m)*(data - m))/len(data)
print(result)

1.25


동형암호버전


HEAAN에서 바로 쓸 수 있도록 FASE로 다시 작성

In [20]:
ctxt = encryptor.encrypt(data)


ctxt_mean = mean(ctxt)
ev.rescale_next(ctxt_mean)
ctxt_mean = algo.put_mask(ctxt_mean, np.arange(ctxt._n_elements))
ev.rescale_next(ctxt_mean)
ev.mod_down_to(ctxt, ctxt_mean.logq)
sub = ev.sub(ctxt, ctxt_mean)
squared = ev.mult(sub, sub) # inplace=False
ev.rescale_next(squared)
summed_eq = sum_reduce(squared)

n = encoder.encode([len(data)]*ctxt.nslots) 
res = ev.div_by_plain(summed_eq, n)

In [22]:
decryptor.decrypt(res)

array([1.25, 1.25, 1.25, ..., 1.25, 1.25, 1.25])

In [24]:
# 스케일 (logp), noise budget(logq) 확인용
ctxt = encryptor.encrypt(data)
ctxt_mean = mean(ctxt)
check(ctxt_mean)
ev.rescale_next(ctxt_mean)
check(ctxt_mean)
ctxt_mean = algo.put_mask(ctxt_mean, np.arange(ctxt._n_elements))
check(ctxt_mean)
ev.rescale_next(ctxt_mean)
check(ctxt)
check(ctxt_mean)
print("ctxt와 ctxt_mean의 logq (noise budget, level)을 맞춰주어야함")
ev.mod_down_to(ctxt, ctxt_mean.logq)
check(ctxt)
check(ctxt_mean)
sub = ev.sub(ctxt, ctxt_mean)
check(sub)
squared = ev.mult(sub, sub) # inplace=False
check(squared)
ev.rescale_next(squared)
check(squared)
summed_eq = sum_reduce(squared)
check(summed_eq)
n = encoder.encode([len(data)]*ctxt.nslots) 
res = ev.div_by_plain(summed_eq, n)
check(res)

Check scale: 60,  noise budget:150
Check scale: 30,  noise budget:120
Check scale: 60,  noise budget:120
Check scale: 30,  noise budget:150
Check scale: 30,  noise budget:90
ctxt와 ctxt_mean의 logq (noise budget, level)을 맞춰주어야함
Check scale: 30,  noise budget:90
Check scale: 30,  noise budget:90
Check scale: 30,  noise budget:90
Check scale: 60,  noise budget:90
Check scale: 30,  noise budget:60
Check scale: 30,  noise budget:60
Check scale: 60,  noise budget:60


In [69]:
decryptor.decrypt(res)

array([1.25, 1.25, 1.25, ..., 1.25, 1.25, 1.25])

In [70]:
np.var(data)

1.25

# RSB data

분석 내용은 stats_fhe repository 참조

In [25]:
## 전체 중에 16384개만 사용
nslots = 2**14 

In [27]:
import csv
import pandas as pd

In [28]:

fn = 'C:/Users/etri/Dropbox/Deepinsight/2022/2022FHE/방문연구/다운로드/통계청/2018년도 SBR(사업자등록번호 기준) 재현자료.csv'